In [ ]:
#imports

In [28]:
#load small input
input = open("./../../data/day_20/small_input.txt", "r").readlines()

In [31]:
#load small2 input
input = open("./../../data/day_20/small2_input.txt", "r").readlines()

In [34]:
#load input
input = open("./../../data/day_20/input.txt", "r").readlines()

In [35]:
#process input
modules = {}
for line in input:
    line = line.strip()
    name,targets = line.split("->")
    name = name.strip()
    if name == "broadcaster":
        typ = "bc"
    else:
        typ = name[:1]
        name = name[1:]
    targets = targets.strip().replace(" ", "").split(",")
    match typ:
        case "bc": modules[name] = {"type":typ,"targets":targets}
        case "%": modules[name] = {"type":typ,"targets":targets,"state":False}
        case "&": modules[name] = {"type":typ,"targets":targets,"state":{}}
#set initial state for conjunction modules
for module in modules.keys():
    if modules[module]["type"] == "&":
        in_connections = {}
        for km,m in modules.items():
            if module in m["targets"]:
                in_connections[km] = False
        modules[module]["state"] = in_connections

In [46]:
#part 1
def reset_states():
    for k,m in modules.items():
        if m["type"] == "%":
            m["state"] = False
        elif m["type"] == "&":
            for k2 in m["state"].keys():
                m["state"][k2] = False

def compare_states(a,b):
    for (ak,av),(bk,bv) in zip(a.items(),b.items()):
        if av != bv:
            print(f"Error: Keys are not the same")
        match av["type"]:
            case "%":
                if av["state"] != bv["state"]: return False
            case "&":
                for (ak2,av2),(bk2,bv2) in zip(av["state"].items(),bv["state"].items()):
                    if av2 != bv2:
                        print(f"Error: Keys are not the same in state")
                    if av2 != bv2: return False
            case _: continue
    return True

def copy_state(state):
    copy = {}
    for key,module in state.items():
        copy_module = {}
        copy_module["type"] = module["type"]
        copy_module["targets"] = [t for t in module["targets"]]
        if module["type"] == "%":
            copy_module["state"] = module["state"]
        elif module["type"] == "&":
            copy_module["state"] = {k:v for k,v in module["state"].items()}
        copy[key] = copy_module
    return copy

def dict_to_tuple(d):
    t = ()
    for k,v in d.items():
        if type(v) == dict:
            t = (*t,(k,dict_to_tuple(v)))
        elif type(v) == list:
            t = (*t,(k,tuple(v)))
        else:
            t = (*t,(k,v))
    return t

## process pulses
def process_button_pulse(modules):
    low_pulses = 0
    high_pulses = 0
    rx_l = 0
    rx_h = 0
    rx_parent = []
    pulses = [("button","broadcaster",False)] #pulses are tuple (source,target,strength)
    while pulses:
        source,target,strength = pulses.pop(0)
        #print(f"Pulse from {source} to {target} with strength {'high' if strength else 'low'}")
        if strength:
            high_pulses += 1
        else:
            low_pulses += 1
        module = modules[target] if target in modules else None
        if not module:
            if target == "rx":
                if strength:
                    rx_h += 1
                    rx_parent.append(source)
                else:
                    rx_l += 1
                    rx_parent.append(source)
            continue
        match module["type"]:
            case "bc":
                pulses.extend( (target,new_target,strength) for new_target in module["targets"] )
            case "%":
                if not strength:
                    module["state"] = not module["state"]
                    pulses.extend( (target,new_target,module["state"]) for new_target in module["targets"] )
            case "&":
                module["state"][source] = strength
                if all(module["state"].values()):
                    pulses.extend( (target,new_target,False) for new_target in module["targets"] )
                else:
                    pulses.extend( (target,new_target,True) for new_target in module["targets"] )
    return modules,low_pulses,high_pulses,(rx_l,rx_h,tuple(rx_parent))

reset_states()
m = copy_state(modules)
sum_lp = 0
sum_hp = 0
for i in range(1000):
    m,lp,hp,_ = process_button_pulse(m)
    sum_lp += lp
    sum_hp += hp

print(f"Part 1: {sum_lp} low pulses and {sum_hp} high pulses -> {sum_lp*sum_hp} pulses")


Part 1: 18361 low pulses and 44384 high pulses -> 814934624 pulses


In [54]:
###part 2: I think this type of problem is terrible, therefore I am going to do a very dirty solution here
#part 1
def reset_states():
    for k,m in modules.items():
        if m["type"] == "%":
            m["state"] = False
        elif m["type"] == "&":
            for k2 in m["state"].keys():
                m["state"][k2] = False

def compare_states(a,b):
    for (ak,av),(bk,bv) in zip(a.items(),b.items()):
        if av != bv:
            print(f"Error: Keys are not the same")
        match av["type"]:
            case "%":
                if av["state"] != bv["state"]: return False
            case "&":
                for (ak2,av2),(bk2,bv2) in zip(av["state"].items(),bv["state"].items()):
                    if av2 != bv2:
                        print(f"Error: Keys are not the same in state")
                    if av2 != bv2: return False
            case _: continue
    return True

def copy_state(state):
    copy = {}
    for key,module in state.items():
        copy_module = {}
        copy_module["type"] = module["type"]
        copy_module["targets"] = [t for t in module["targets"]]
        if module["type"] == "%":
            copy_module["state"] = module["state"]
        elif module["type"] == "&":
            copy_module["state"] = {k:v for k,v in module["state"].items()}
        copy[key] = copy_module
    return copy

def dict_to_tuple(d):
    t = ()
    for k,v in d.items():
        if type(v) == dict:
            t = (*t,(k,dict_to_tuple(v)))
        elif type(v) == list:
            t = (*t,(k,tuple(v)))
        else:
            t = (*t,(k,v))
    return t

## process pulses
def process_button_pulse(modules):
    low_pulses = 0
    high_pulses = 0
    nr_parent_low = []
    nr_parent_high = []
    pulses = [("button","broadcaster",False)] #pulses are tuple (source,target,strength)
    while pulses:
        source,target,strength = pulses.pop(0)
        #print(f"Pulse from {source} to {target} with strength {'high' if strength else 'low'}")
        if strength:
            high_pulses += 1
        else:
            low_pulses += 1
        module = modules[target] if target in modules else None
        if not module:
            continue

        ###space for dirty code
        if target == "nr":
                if strength:
                    nr_parent_high.append(source)
                else:
                    nr_parent_low.append(source)
        ###

        match module["type"]:
            case "bc":
                pulses.extend( (target,new_target,strength) for new_target in module["targets"] )
            case "%":
                if not strength:
                    module["state"] = not module["state"]
                    pulses.extend( (target,new_target,module["state"]) for new_target in module["targets"] )
            case "&":
                module["state"][source] = strength
                if all(module["state"].values()):
                    pulses.extend( (target,new_target,False) for new_target in module["targets"] )
                else:
                    pulses.extend( (target,new_target,True) for new_target in module["targets"] )
    return modules,low_pulses,high_pulses,(tuple(nr_parent_low),tuple(nr_parent_high))

reset_states()


In [55]:
# part 2
m = modules
rxs = []
lh = []
mm = []
ff = []
fk = []
for i in range(100000):
    m,lp,hp,(nr_l,nr_h) = process_button_pulse(m)
    for n in nr_h:
        match n:
            case "lh": lh.append(i)
            case "mm": mm.append(i)
            case "ff": ff.append(i)
            case "fk": fk.append(i)
            case _: print(f"Did not expect to find {n}")

print(f"lh:{lh}\nmm:{mm}\nff:{ff}\nfk:{fk}")
#lh repeats every 3761
#mm repeats every 3919
#ff repeats every 3797
#fk repeats every 4079
# -> calculated lcm to 228282646835717

lh:[3760, 7521, 11282, 15043, 18804, 22565, 26326, 30087, 33848, 37609, 41370, 45131, 48892, 52653, 56414, 60175, 63936, 67697, 71458, 75219, 78980, 82741, 86502, 90263, 94024, 97785]
mm:[3918, 7837, 11756, 15675, 19594, 23513, 27432, 31351, 35270, 39189, 43108, 47027, 50946, 54865, 58784, 62703, 66622, 70541, 74460, 78379, 82298, 86217, 90136, 94055, 97974]
ff:[3796, 7593, 11390, 15187, 18984, 22781, 26578, 30375, 34172, 37969, 41766, 45563, 49360, 53157, 56954, 60751, 64548, 68345, 72142, 75939, 79736, 83533, 87330, 91127, 94924, 98721]
fk:[4078, 8157, 12236, 16315, 20394, 24473, 28552, 32631, 36710, 40789, 44868, 48947, 53026, 57105, 61184, 65263, 69342, 73421, 77500, 81579, 85658, 89737, 93816, 97895]


In [ ]:
#run part 1 logic with loop detection; buggy, but apparently not necessary anyways
cache = {}
cache2 = {}
sum_lp, sum_hp = 0,0
current = modules
for i in range(1000):
    current = copy_state(current)
    current,lp,hp = process_button_pulse(current)
    t = dict_to_tuple(current)
    if t in cache:
        j,lp,hp = cache[t]
        print(f"Loop detected between iteration {j} and {i}")

        loop_length = i - j
        loop_lp = lp
        loop_hp = hp
        for k in range(j+1,i):
            l,h = cache2[k]
            loop_lp += l
            loop_hp += h
        break
    sum_lp += lp
    sum_hp += hp
    cache[t] = (i,lp,hp)
    cache2[i] = (lp,hp)

print(f"Processed until index {i} with lp {sum_lp} and hp {sum_hp}")

loop_amount = (1000-i) // loop_length
sum_lp += loop_amount * loop_lp
sum_hp += loop_amount * loop_hp
print(f"Loop of length {loop_length} which fits {loop_amount} times with lp {loop_lp} and hp {loop_hp}, lp {sum_lp} and hp {sum_hp}")
remaining_iterations = (1000-i) % loop_length
for i in range(remaining_iterations):
    current = copy_state(current)
    current,lp,hp = process_button_pulse(current)
    sum_lp += lp
    sum_hp += hp
print(f"Processed {remaining_iterations} remaining iterations with lp {sum_lp} and hp {sum_hp}")

result = sum_lp*sum_hp
print(f"Part 1: {result}")
    